In [1]:
import pyzx as zx
import random
from qiskit import QuantumCircuit
from qiskit.circuit import QuantumCircuit, ParameterVector
import csv
import time

In [3]:
def generate_H_S_CNOT_T_circuit(qubits, gates, p_t=0, seed=1000):
    random.seed(seed)  
    p_s = 0.333 * (1.0 - p_t)  
    p_had = 0.333 * (1.0 - p_t)  
    p_cnot = 0.333 * (1.0 - p_t)  

    c = zx.Circuit(qubits) 
    for _ in range(gates):
        r = random.random() 
        if r < p_had:
            c.add_gate("HAD", random.randrange(qubits))
        elif r < p_had + p_s:
            c.add_gate("S", random.randrange(qubits))
        elif r < p_had + p_s + p_t:
            c.add_gate("T", random.randrange(qubits))
        else:
            tgt = random.randrange(qubits)
            while True:
                ctrl = random.randrange(qubits)
                if ctrl != tgt:
                    break
            c.add_gate("CNOT", tgt, ctrl)
    return c

In [5]:
# seed固定
seed = 1000
random.seed(seed)

# パラメータ設定
qubits_list = [4, 8, 12, 16]
p_t_lists = [i * 0.04 for i in range(0, 11)]
gate_num_lists = [100, 500, 1000, 2000, 5000, 10000, 50000,100000]

# CSV書き込みの準備
output_file = "../kekka/datasets.csv"
fieldnames = [
    "data_name","qubits", "gates", "p_t","seeds","depth","tcount","H","S","CNOT","time"
]

with open(output_file, "w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()

    for qubit in qubits_list:
        for gate_num in gate_num_lists:
            for p_t in p_t_lists:
                try:
                    start = time.time()
                    # 回路作成
                    c = generate_H_S_CNOT_T_circuit(qubits = qubit,gates= gate_num,p_t = p_t,seed = seed)
                    #circuit_stats = c.stats_dict()

                    #qiskit変換
                    circuit_qiskit = QuantumCircuit.from_qasm_str(c.to_basic_gates().to_qasm())

                    depth = circuit_qiskit.depth()
                    gate_count_dict = circuit_qiskit.count_ops()
                    #print("cx",gate_count_dict["cx"] , "h",gate_count_dict["h"], "s",gate_count_dict["s"], "t",gate_count_dict["t"])

                    end = time.time()

                    #dataname = "q" + str(qubit) + "_gates" + str(gate_num) + "_pt" +str(p_t)+ "_seeds" + str(seed)
                    # 結果をCSVに書き込む
                    writer.writerow({
                        "data_name": "q" + str(qubit) + "_gates" + str(gate_num) + "_pt" +str(p_t)+ "_seeds" + str(seed),
                        "qubits": qubit,
                        "gates": gate_num,
                        "p_t": p_t,
                        "seeds": seed,
                        "depth": depth,
                        "tcount": gate_count_dict.get("t","0"),
                        "H": gate_count_dict["h"],
                        "S": gate_count_dict["s"],
                        "CNOT": gate_count_dict["cx"],
                        "time": end-start
                    })

                    print(f"done. Qubits={qubit}, Gate_Num={gate_num}, P_T={p_t},seed={seed},time={end-start}")

                except Exception as e:
                    print(f"Error for Qubits={qubit}, Gate_Num={gate_num}, P_T={p_t}: {e}")

done. Qubits=4, Gate_Num=100, P_T=0.0,seed=1000,time=0.004080057144165039
done. Qubits=4, Gate_Num=100, P_T=0.04,seed=1000,time=0.000873565673828125
done. Qubits=4, Gate_Num=100, P_T=0.08,seed=1000,time=0.0006775856018066406
done. Qubits=4, Gate_Num=100, P_T=0.12,seed=1000,time=0.0007183551788330078
done. Qubits=4, Gate_Num=100, P_T=0.16,seed=1000,time=0.0006425380706787109
done. Qubits=4, Gate_Num=100, P_T=0.2,seed=1000,time=0.0008749961853027344
done. Qubits=4, Gate_Num=100, P_T=0.24,seed=1000,time=0.0006117820739746094
done. Qubits=4, Gate_Num=100, P_T=0.28,seed=1000,time=0.0007312297821044922
done. Qubits=4, Gate_Num=100, P_T=0.32,seed=1000,time=0.0007672309875488281
done. Qubits=4, Gate_Num=100, P_T=0.36,seed=1000,time=0.0006868839263916016
done. Qubits=4, Gate_Num=100, P_T=0.4,seed=1000,time=0.0007202625274658203
done. Qubits=4, Gate_Num=500, P_T=0.0,seed=1000,time=0.0030739307403564453
done. Qubits=4, Gate_Num=500, P_T=0.04,seed=1000,time=0.003031015396118164
done. Qubits=4, Gat